#Trabalho de Mineração de Dados (PI)

#1. Import de Database e Bibliotecas:

In [37]:
!git clone https://github.com/nnieseeghosts/supermarket.git

import pandas as pd
import numpy as np
import random
import itertools

fatal: destination path 'supermarket' already exists and is not an empty directory.


#2. Import do supermarket:

In [38]:
sup = pd.read_csv(filepath_or_buffer='/content/supermarket/supermarket.csv')

#3. Regras de Associação Apriori:

#3.1. Definição de limiares:

In [58]:
minSuporte = 0.2  # Limiar mínimo de suporte
qtd_ant = 4 # Quantidade de antecedentes
minConfianca = 0.75 #Limiar mínimo de Confiança
minLift = 1.01  # Limiar mínimo de lift
minConv = 1.01  # Limiar mínimo de métrica extra


#3.2. Cálculo dos Suportes:

In [40]:
#Cálculo dos suportes de todas as colunas
suporte = (sup == 't').mean()
suporte

department1      0.226281
department2      0.028312
department3      0.019451
department4      0.018154
department5      0.037821
                   ...   
department213    0.004755
department214    0.000000
department215    0.000000
department216    0.000000
total            0.000000
Length: 217, dtype: float64

In [41]:
#Aqui eu separo as colunas com valor superior ao limiar de suporte pré definido
colunaslimiar =  suporte[suporte >= minSuporte].index.tolist()
colunaslimiar[0]

'department1'

#3.3. Geração de Regras:

#3.3.1. Utilzando a Biblioteca Itertools:

In [42]:
comb_iter = itertools.combinations(colunaslimiar, qtd_ant) #Uso a itertools para criar todas as combinações

dict_ant_iter = {f'antecedente{i+1}': list(comb) for i, comb in enumerate(comb_iter)} #Salvo todos os valores criados em uma biblioteca

#3.3.2. Utilizando a Biblioteca Random:

In [59]:
regras_geradas = 100 #Esta variável define a quantidade de regras que serão testadas
regra = {} #Define a biblioteca regra

for i in range(regras_geradas): #Faço um for de i pela quantidade de regras geradas
  col_aleatoria = random.sample(colunaslimiar, qtd_ant) #Uso a biblioteca random para gerar regras aleatórias
  regra[f'regra{i}'] = col_aleatoria #Salvo na biblioteca


PS: De acordo com testes, o uso de regras aleatórias se torna mais eficiente neste trabalho em específico, pois a geração de todas as combinações de regra com antecedentes com quantidade de colunas superior a 2 cresce potencialmente, então se gerarmos todas as combinações de antecedentes com 4 colunas utilizand o itertools, obteríamos 1.413.720 combinações, estragando toda a performance do código.

#3.4. Regras de Associação Apriori:

#3.4.1. Confiança:

A confiança é um número que expressa a possibilidade de um item ser comprado quando outro item correlato é comprado. Por exemplo, qual a confiança que um cliente irá comprar queijo considerando que ele já comprou Pão e margarina.   

A confiança é calculada através da seguinte equação:


conf(X→Y) = supp(X U Y) / supp(X)

Fonte: (https://www.computersciencemaster.com.br/como-funciona-o-algoritmo-apriori/)

#3.4.2. Lift:

Esta medida calcula também a possibilidade de um item ser comprado em relação a outro item. Porém, esta medida considera a popularidade de ambos os itens.

lift(X→Y) = supp(X U Y) / supp(X) * supp(Y)

Neste cálculo podemos analisar da seguinte forma: se o valor de lift(x→y) > 1 existe uma relação de compra entre estes dois itens. Já se o valor de lift(x→y) < 1 é muito provável que não exista uma relação clara expressa no dataset.

Fonte: (https://www.computersciencemaster.com.br/como-funciona-o-algoritmo-apriori/)

#3.4.3. Convicção:

Outra medida que pode ser calculada é a convicção:

conv (x → y) = 1 – supp(y) / 1 – conf(x → y)

Essa medida expressa a convicção pode ser interpretado como a razão da freqüência esperada que X ocorre sem Y (isto é, a freqüência que a regra faz uma predição incorreta) se X e Y fossem independentes divididos pela freqüência observada de predições incorretas.

Fonte: (https://www.computersciencemaster.com.br/como-funciona-o-algoritmo-apriori/)


#3.5. Cálculo e Geração de Regras Fortes:

In [60]:
def combinacoes_3(sequencia):
    combinacoes = []
    for i in range(1, 3):
        for j in range(len(sequencia) - i + 1):
            combinacoes.append(sequencia[j:j+i])
    combinacoesunicas = set(combinacoes)
    return list(combinacoesunicas)

def combinacoes_4(sequencia):
    combinacoes = []
    for i in range(1, 4):
        for j in range(len(sequencia) - i + 1):
            combinacoes.append(sequencia[j:j+i])
    combinacoesunicas = set(combinacoes)
    return list(combinacoesunicas)

In [45]:
antecedente = regra.get(f'regra{1}')

In [46]:
sequencia = tuple(antecedente)
combinacoes = combinacoes_3(sequencia)

listacon = []

for combinacao in combinacoes:
    listacon.append(combinacao)
print(listacon)

[('cleaners-polishers',), ('cleaners-polishers', "'canned fruit'"), ("'canned fruit'",)]


In [47]:
regras_fortes_2 = set() #Crio listas de regras fortes com o set de 2 colunas, para garantir que os resultados listados não irão se repetir
regras_fortes_3 = set() #Crio listas de regras fortes com o set de 3 colunas, para garantir que os resultados listados não irão se repetir
regras_fortes_4 = set() #Crio listas de regras fortes com o set de 4 colunas, para garantir que os resultados listados não irão se repetir


In [61]:
if 1< qtd_ant <= 4:  #O código só roda se o valor de colunas de antecedentes for até 4
  for i in range (1, len(regra)): #Faz um for do tamanho da quantidade de regras geradas anteriormente
    #Cálculo do suporte das colunas definidas
    antecedente = regra.get(f'regra{i}') #Pega o valor da regra e transforma na variável antecedente
    consequente = tuple(antecedente)
    if qtd_ant == 3:
      combinacoes = combinacoes_3(consequente)
    if qtd_ant == 4:
      combinacoes = combinacoes_4(consequente)
    listacon = []
    for combinacao in combinacoes:
      listacon.append(combinacao)
    listacon = [list(x) for x in listacon]
    if qtd_ant == 2:
      listacon = [antecedente[0], antecedente[1]]
    for x in range (len(listacon)): #Dentro da quantidade de antecedentes, faz outro for para rodar por dentro deste valor
      sup_antecedente = (sup[antecedente] == 't').all(axis=1).mean() #Calculo o suporte do antecedente
      if qtd_ant != 2:
        sup_consequente = (sup[listacon[x]] == 't').all(axis=1).mean() #Calculo o suporte do consequente
      else:
        sup_consequente = (sup[listacon[x]] == 't').mean() #Calculo o suporte do consequente

      #Confiança
      if sup_antecedente == 0: #Se o suporte for 0, a confiança será 0, isso não ocorrerá pois só usaremos as colunas que estavam no limiar anterior, mas fiz isso para evitar erros
        confianca = 0
      else:
        confianca = sup_antecedente / sup_consequente #Caso não seja 0, aqui se calcula a confiança das colunas

      confiancap = round(confianca * 100, 2) #Arredondamos o valor e transformamos em porcentagem, a fim de facilitar o entendimento

      if confianca > minConfianca: #Se a confiança estiver de acordo com o limiar a regra continua
        consequente_lift = 1 #Criamos uma variável para calcular o lift e damos o valor de 1 para não inteferir a multiplicação
        #A utilidade dessa variável é permitir que possamos escolher qtd_antes e fazer o código funcionar sem recisar alterá-lo
        ant_lift = antecedente.copy() #Copio a lista de antecedentes
        for y in range(qtd_ant):
          if antecedente[y] in listacon[x]: #Se a coluna já estiver na regra de consequente gerada anteriormente
            ant_lift.remove(antecedente[y]) #Ele exclui as colunas

        consequente_lift = sup_consequente * (sup[ant_lift] == 't').all(axis=1).mean() #consequente_lift é igual ao próprio valor multiplicado pelo suporte da próxima coluna, isso armazena todas as multiplicações

          #Lift
        if sup_antecedente == 0: #Caso dê algum erro e o valor do suporte seja 0, lift será 0
          lift = 0
        else:
          lift = sup_antecedente / consequente_lift #Caso contrário, será realizado o cálculo do lift

        if lift > minLift: #Se Lift estiver de acordo com o limiar, a regra continua sendo calculada

          #Convicção
          suporte_conv = (sup[ant_lift] == 't').all(axis=1).mean() #Calculo o suporte desta lista

          if suporte_conv == 0: #Caso dê algum erro e o valor seja 0, convicção será 0
            conviccao = 0
          else:
            conviccao = (1 - suporte_conv) / (1 - confianca) #Caso contrário, a confiança será calculada

          if conviccao > minConv: #Se o confiança estar de acordo com o limiar, essa relação será uma regra forte!
            print(f'A regra gerada foi {antecedente} =>{listacon[x]}')
            print(f'Sua confiança foi de: {confiancap}%')
            print(f'O Lift calculado obteve resultado de: {lift:.2f}')
            print(f'A convicção da regra é de {conviccao:.2f}')
            print('')
            print(f'Com isso, podemos dizer que é uma regra forte, pois obteve associações superiores aos limiares definidos.')
            print('')

            if qtd_ant == 2 and antecedente: #Se a quantidade de colunas antecedentes for 2:
              regras_fortes_2.add(f'{antecedente} =>{listacon[x]}') #Salvo a regra forte no set em formato de tupla
            if qtd_ant == 3 and antecedente != listacon[x]: #Se a quantidade de colunas antecedentes for 3:
              regras_fortes_3.add(f'{antecedente} =>{listacon[x]}') #Salvo a regra forte no set em formato de tupla
            if qtd_ant == 4 and antecedente != listacon[x]: #Se a quantidade de colunas antecedentes for 4:
              regras_fortes_4.add(f'{antecedente} =>{listacon[x]}') #Salvo a regra forte no set em formato de tupla
else:
  print("Quantidade de antecedentes superior a 4 ou inferior a 2!") #Printo isto cajo a quantidade de colunas definidas anteriormente for maior que 4 ou menor que 2




A regra gerada foi ["'bread and cake'", 'fruit', "'laundry needs'", 'wrapping'] =>['fruit', "'laundry needs'", 'wrapping']
Sua confiança foi de: 83.6%
O Lift calculado obteve resultado de: 1.16
A convicção da regra é de 1.71

Com isso, podemos dizer que é uma regra forte, pois obteve associações superiores aos limiares definidos.

A regra gerada foi ['department1', 'biscuits', 'jams-spreads', "'frozen foods'"] =>['department1', 'biscuits', 'jams-spreads']
Sua confiança foi de: 75.0%
O Lift calculado obteve resultado de: 1.28
A convicção da regra é de 1.65

Com isso, podemos dizer que é uma regra forte, pois obteve associações superiores aos limiares definidos.

A regra gerada foi ['vegetables', 'department137', 'juice-sat-cord-ms', "'bread and cake'"] =>['vegetables', 'department137', 'juice-sat-cord-ms']
Sua confiança foi de: 82.38%
O Lift calculado obteve resultado de: 1.14
A convicção da regra é de 1.59

Com isso, podemos dizer que é uma regra forte, pois obteve associações superior

#4. Conclusões:

#4.1. Conclusões de desconto:

De acordo com conhecimento empírico, podemos concluir que setores com 75% de compra são considerados presentes em todos os carrinhos, e uma promoção não iria fazer tanta diferença, já que mesmo sem promoção o produto vende na grande maioria dos casos.

E setores com menos de 40% de venda são considerados dependentes e/ou não tão escolhidos, então acredito que setores com valores superiores a 40% e até 60% de conversão em venda são os ideais para serem colocados em desconto, isto é, promoções singulares (Do tipo "Em compras acima de 2 produtos", desconto de preços convencionais, etc) que não dependem de outro produto para entrar em desconto.

In [49]:
minSuporteConclusao = 0.4 #Defino o limiar mínimo
maxSuporteConclusao = 0.6 #Defino o limiar máximo

for a in range (len(colunaslimiar)): #Em a no tamanho da variável colunas limiar:
  sup_conclusao = (sup[colunaslimiar[a]] == 't').mean() #Calcula o suporte das colunas
  if maxSuporteConclusao >= sup_conclusao >= minSuporteConclusao: #Separa os suportes dentro dos limiares
    print(f'Seria de interesse do mercado a promoção do produto/setor {colunaslimiar[a]}') #Printa quais produtos/setores deveriam ser descontados

Seria de interesse do mercado a promoção do produto/setor juice-sat-cord-ms
Seria de interesse do mercado a promoção do produto/setor biscuits
Seria de interesse do mercado a promoção do produto/setor 'breakfast food'
Seria de interesse do mercado a promoção do produto/setor sauces-gravy-pkle
Seria de interesse do mercado a promoção do produto/setor 'frozen foods'
Seria de interesse do mercado a promoção do produto/setor 'pet foods'
Seria de interesse do mercado a promoção do produto/setor 'party snack foods'
Seria de interesse do mercado a promoção do produto/setor 'tissues-paper prd'
Seria de interesse do mercado a promoção do produto/setor 'soft drinks'
Seria de interesse do mercado a promoção do produto/setor cheese
Seria de interesse do mercado a promoção do produto/setor margarine
Seria de interesse do mercado a promoção do produto/setor department137


Especificamente falando:

* Na compra de 2 pacotes da mesma marca na seção 'biscuits', a terceira sai com 20% de desconto (Incentivando a compra de pelo menos 3 pacotes, assim como no atacado)
* A partir de 300g de queijo o valor sai com 10% de desconto.
* E em setores de compra de unidades de produto como 'juice-sat-cord-ms', -tissues-paper prd' e outros, caberia uma promoção generalizada dos produtos.

#4.2. Conclusões de setores de alto retorno:

In [50]:
minSupAltoRetorno = 0.636 #Defino um limiar com o intuito de obter apenas os produtos de alta conversão

for a in range (len(colunaslimiar)): #Para a no tamanho da variável colunaslimiar
  sup_conclusao = (sup[colunaslimiar[a]] == 't').mean() #Calculo o suporte das colunas
  if sup_conclusao >= minSupAltoRetorno: #Se estiver dentro do limiar
    print(f'Seria de interesse do mercado a localização estratégica do produto/setor {colunaslimiar[a]}') #Printo os setores de alta conversão

Seria de interesse do mercado a localização estratégica do produto/setor 'bread and cake'
Seria de interesse do mercado a localização estratégica do produto/setor fruit
Seria de interesse do mercado a localização estratégica do produto/setor vegetables


Assim como na maioria dos mercados, as padarias e hortifruits são sempre localizadas no final do supermerdado, pois como produtos deste setor são sempre os mais comprados, os clientes vão ir de qualquer jeito até lá, então quanto maior o caminho, mais produtos eles verão.

#4.3. Conclusões com listas de regras fortes:

#4.3.1. Regras de 2 colunas

In [51]:
regras_fortes_2

{'["\'baking needs\'", "\'bread and cake\'"] =>\'baking needs\'',
 '["\'bread and cake\'", \'sauces-gravy-pkle\'] =>sauces-gravy-pkle',
 '["\'canned vegetables\'", "\'bread and cake\'"] =>\'canned vegetables\'',
 '["\'pet foods\'", "\'bread and cake\'"] =>\'pet foods\'',
 '[\'margarine\', "\'bread and cake\'"] =>margarine',
 '[\'wrapping\', "\'bread and cake\'"] =>wrapping'}

Como o setor 'bread and cake' foi o único que apareceu em todas, da para concluir que os outros itens são comprados em conformidade com a compra deste item.

Com isso podemos concluir que:
* Como a compra de pãe já é de alta conversão, em comprar a partir de 6 pães ou o equivalente em gramas, categorias como 'jams-spreads' terão desconto de 10%
* Como a margarina em nossa cultura ja é bastante utilizada com pães, ao invés de desconto, seria interessante deixá-los próximos aos pães, bem como itens da categoria 'small goods'.

#4.3.2. Regras com 3 colunas:

In [62]:
regras_fortes_3

{'["\'baking needs\'", \'cleaners-polishers\', \'department1\'] =>[\'cleaners-polishers\', \'department1\']',
 '["\'baking needs\'", \'sauces-gravy-pkle\', \'margarine\'] =>[\'sauces-gravy-pkle\', \'margarine\']',
 '["\'bread and cake\'", \'department1\', "\'small goods2\'"] =>[\'department1\', "\'small goods2\'"]',
 '["\'canned fruit\'", \'deodorants-soap\', "\'baking needs\'"] =>["\'canned fruit\'", \'deodorants-soap\']',
 '["\'dental needs\'", "\'breakfast food\'", "\'baking needs\'"] =>["\'dental needs\'", "\'breakfast food\'"]',
 '["\'dental needs\'", \'beef\', "\'bread and cake\'"] =>["\'dental needs\'", \'beef\']',
 '["\'dental needs\'", \'department122\', \'vegetables\'] =>["\'dental needs\'", \'department122\']',
 "['cleaners-polishers', 'department137', 'vegetables'] =>['cleaners-polishers', 'department137']",
 '[\'deodorants-soap\', \'margarine\', "\'baking needs\'"] =>[\'deodorants-soap\', \'margarine\']',
 '[\'jams-spreads\', \'cleaners-polishers\', "\'bread and cake\'"] =

Como a maioria das associações são relacionadas aos conjuntos de itens de Padaria e Hortifruit, seria interessante o posicionamento estratégico dos setores não só no fim do mercado, mas com distâncias entre si também, por exeplo:

* Enquanto a entrada fica na posição inferior centrar do supermercado, o Hortifruit fica na posição superior esquerda e a Padaria na posição superior direita, fazendo com que os clientes transitem mais ainda pelo mercado, aumentando a compra de diversos outros itens que também já foram colocados em promoção nas propostas anteriores.

#4.3.3. Regras com 4 colunas:

In [63]:
regras_fortes_4

{'["\'baking needs\'", "\'tissues-paper prd\'", "\'small goods2\'", \'vegetables\'] =>["\'baking needs\'", "\'tissues-paper prd\'", "\'small goods2\'"]',
 '["\'baking needs\'", "\'tissues-paper prd\'", "\'small goods2\'", \'vegetables\'] =>["\'tissues-paper prd\'", "\'small goods2\'", \'vegetables\']',
 '["\'baking needs\'", \'department137\', \'wrapping\', \'margarine\'] =>[\'department137\', \'wrapping\', \'margarine\']',
 '["\'bread and cake\'", \'confectionary\', "\'tissues-paper prd\'", "\'small goods2\'"] =>[\'confectionary\', "\'tissues-paper prd\'", "\'small goods2\'"]',
 '["\'bread and cake\'", \'fruit\', "\'laundry needs\'", \'wrapping\'] =>[\'fruit\', "\'laundry needs\'", \'wrapping\']',
 '["\'canned fish-meat\'", "\'canned vegetables\'", \'cleaners-polishers\', \'vegetables\'] =>["\'canned fish-meat\'", "\'canned vegetables\'", \'cleaners-polishers\']',
 '["\'dairy foods\'", "\'small goods2\'", "\'frozen foods\'", "\'bread and cake\'"] =>["\'dairy foods\'", "\'small goods2\

Bem como citado anteriormente, se sobressai a relação destes setores nos carrinhos, mas com a relação de 'canned fruit' podemos dizer que também seria interessantes a realocação deste setor entre o Hortifruit e a Padaria, e além disso descontar seu valor unitariamente ou com relação a comprar em 'bread and cake', já que o uso de frutas frescas ou enlatadas é sim relacionado com a compra de bolos.